<a href="https://colab.research.google.com/github/jonG312/Orca-Model/blob/main/orca_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git clone https://github.com/tloen/llama-int8 && cd llama-int8

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!safetensors==0.3.1

In [ ]:
!pip install  datasets==2.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00


In [ ]:
!pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.4 MB/s eta 0:00:00


In [ ]:
!pip install  triton==2.0.0

In [ ]:
!pip install  texttable

In [ ]:
!pip install toml

In [ ]:
!pip install numpy

In [ ]:
!pip install protobuf

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

In [ ]:
# Hugging Face model_path
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#generate text function
def generate_text(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    return f'[!] Response: {string}'

# Sample Test Instruction Used by Youtuber Sam Witteveen https://www.youtube.com/@samwitteveenai
system = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.'
instruction = 'Write a letter to Sam Altman, CEO of OpenAI, requesting him to convert GPT4 a private model by OpenAI to an open source project'
print(generate_text(system, instruction))

[!] Response: Dear Sam Altman,

I am writing to you regarding the ongoing controversy surrounding GPT4 and its lack of transparency. As a leader in the AI community, I believe it is essential to ensure that AI models such as GPT4 are available and accessible to all.

I understand that GPT4 is a proprietary model developed by OpenAI and that its conversion to an open source project would require significant investment. However, I urge you to consider the potential benefits of doing so.

By converting GPT4 to an open source project, we would be able to gain access to its source code, which would allow us to analyze its performance and improve the state-of-the-art in AI. This would also enable us to contribute to the development of GPT4 and improve its functionality.

In addition, converting GPT4 to an open source project would align with OpenAI's mission of advancing AI in a way that is safe and beneficial to humanity. It would ensure that the benefits of GPT4 are available to everyone, 